In [200]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import ast, os, re, json, requests
from glob import glob
from time import sleep
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
data_folder = ''
external_data_folder = ''

In [231]:
#free-text data
free_text_df = pd.read_csv(data_folder+'data/ade_text_table_onsides_pred.csv')
free_text_df = free_text_df[free_text_df.pt_meddra_id.notna()]
#tabular data
tabular_df = pd.read_csv(data_folder+'data/parsed_ade_tabular.csv')

## concat data

In [232]:
free_text_df.head(1)

,label_id,found_term,meddra_id,location,string,section,set_id,drug,spl_version,pt_meddra_id,pt_meddra_term,Pred0,Pred1
0,extavia,anaemia,10002034,785,anaemia IU Extavia treatment. The most serious...,AR,extavia,NaN,NaN,10002034.0,Anaemia,1.035913,0.0


In [233]:
tabular_df['exact_match_list'] = tabular_df['exact_match_list'].apply(lambda x: ast.literal_eval(x))
tabular_df['matched_codes'] = tabular_df['matched_codes'].apply(lambda x: ast.literal_eval(x))
tabular_df.head(1)

,drug,col,txt,ade,exact_match_list,matched_codes
0,nexavar,Unnamed: 0,Hepatobi- liary disorders hepatitis* Skin and ...,Hepatobiliary disorders hepatitis* Skin and su...,"[skin desquamation, inflammation, mucosal infl...","[10040844, 10061218, 10028116, 10050537, 10054..."


In [234]:
tab_df = tabular_df.explode(['exact_match_list', 'matched_codes'])[['drug', 'exact_match_list', 'matched_codes']]
tab_df.columns = ['drug', 'pt_meddra_term', 'pt_meddra_id']
txt_df = free_text_df[['label_id', 'pt_meddra_term', 'pt_meddra_id']]
txt_df.columns = ['drug', 'pt_meddra_term', 'pt_meddra_id']
print(tab_df.shape[0], txt_df.shape[0])
ade_all_labels = pd.concat([tab_df, txt_df]).drop_duplicates()
print(ade_all_labels.shape[0])
ade_all_labels.head(1)

49629 40986
59791


,drug,pt_meddra_term,pt_meddra_id
0,nexavar,skin desquamation,10040844


In [235]:
ade_all_labels.drug.nunique()

1528

## map drugs to rxnorm

In [198]:
#read in table of drug info (and filter for relevant columns)
df = pd.read_excel(data_folder+'data/Medicines_output_european_public_assessment_reports.xlsx', skiprows=8)
df = df[df.Category == 'Human'][['Medicine name', 'Product number', 'Active substance', 'Authorisation status',
                                 'International non-proprietary name (INN) / common name',
                                 'ATC code']]
df = df[df['Medicine name'].str.lower().isin(ade_all_labels.drug)]
print(df.shape)
df.head(1)

(1538, 6)


,Medicine name,Product number,Active substance,Authorisation status,International non-proprietary name (INN) / common name,ATC code
0,Steglujan,EMEA/H/C/004313,"ertugliflozin l-pyroglutamic acid, sitagliptin...",Authorised,"ertugliflozin, sitagliptin",A10BD24


In [201]:
rxnorm = pd.read_csv(external_data_folder+'athena_rxnorm_atc/CONCEPT.csv', delimiter = '\t')
rxnorm = rxnorm[(rxnorm.domain_id == 'Drug')&(rxnorm.vocabulary_id.isin(['RxNorm', 'RxNorm Extension']))&(rxnorm.concept_class_id.str.contains('Ingredient'))]
rxnorm['concept_name'] = rxnorm['concept_name'].apply(lambda x: x.lower())
rxnorm.head(1)

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
212836,40224159,alcaftadine,Drug,RxNorm,Ingredient,S,1000082,20100905,20991231,NaN


In [205]:
df['rxnorm_code'] = None
for i, row in tqdm(df.iterrows()):
  codes = []
  try:
    name = str(row['International non-proprietary name (INN) / common name'].lower())
    d_df = rxnorm[rxnorm.concept_name == name]
    if d_df.shape[0] > 0:
      codes = d_df.concept_id.tolist()
      df.at[i, 'rxnorm_code'] = codes
  except:
    pass
df['num_ingredients'] = df.rxnorm_code.apply(lambda x: len(x) if x is not None else 0)
df['num_ingredients'].value_counts()

1538it [00:09, 170.52it/s]


1    1101
0     330
2     100
3       5
4       2
Name: num_ingredients, dtype: int64

In [206]:
#first map using umls rxnorm names
rxnorm = pd.read_csv(external_data_folder+'umls_rxnorm.csv')
rxnorm = rxnorm[['CODE', 'STR']]
rxnorm.STR = rxnorm.STR.apply(lambda x: x.lower())
rxnorm_dict = dict(zip(rxnorm.STR, rxnorm.CODE))
rxnorm.head(1)

,CODE,STR
0,1926948,"1,2-dipalmitoylphosphatidylcholine"


In [209]:
#first map on INN name
df['rxnorm_code'] = df.apply(lambda x:
                             list(set([rxnorm_dict[i] for i in x['International non-proprietary name (INN) / common name'].split(', ') if i in rxnorm_dict.keys()]))
                             if x['num_ingredients'] == 0 and str(x['International non-proprietary name (INN) / common name']) != 'nan'
                             else x['rxnorm_code'],
                             axis = 1)
df['rxnorm_code'] = df.apply(lambda x:
                             list(set([rxnorm_dict[i] for i in x['Active substance'].split(', ') if i in rxnorm_dict.keys()]))
                             if x['num_ingredients'] == 0 and str(x['Active substance']) != 'nan'
                             else x['rxnorm_code'],
                             axis = 1)
df['num_ingredients'] = df.rxnorm_code.apply(lambda x: len(x) if x is not None else 0)
df['num_ingredients'].value_counts()

1    1156
2     182
0     179
3      15
4       6
Name: num_ingredients, dtype: int64

In [211]:
for i, row in tqdm(df.iterrows()):
  try:
    if row['num_ingredients'] == 0:
      drug_codes = []
      if str(row['International non-proprietary name (INN) / common name']) != 'nan':
        drugs = row['International non-proprietary name (INN) / common name'].split(', ')
        for drug in drugs:
          if drug in rxnorm_dict.keys():
            drug_codes.append(rxnorm_dict[drug])
          else:
            url = 'https://rxnav.nlm.nih.gov/REST/rxcui.json?name={}&search=1'.format(drug)
            j = requests.get(url).json()
            drug_codes.extend(j['idGroup']['rxnormId'])
      if str(row['Active substance']) != 'nan':
        drugs = row['Active substance'].split(', ')
        for drug in drugs:
          if drug in rxnorm_dict.keys():
            drug_codes.append(rxnorm_dict[drug])
          else:
            url = 'https://rxnav.nlm.nih.gov/REST/rxcui.json?name={}&search=1'.format(drug)
            j = requests.get(url).json()
            drug_codes.extend(j['idGroup']['rxnormId'])
      drug_codes = list(set(drug_codes))
      if len(drug_codes) > 0:
        df.at[i, 'rxnorm_code'] = drug_codes
  except:
      continue

1538it [00:51, 29.73it/s]


In [212]:
df['num_ingredients'] = df.rxnorm_code.apply(lambda x: len(x) if x is not None else 0)
df['num_ingredients'].value_counts()

1    1195
2     201
0     109
3      21
4       7
5       2
7       1
6       1
9       1
Name: num_ingredients, dtype: int64

In [210]:
##################################

In [ ]:
##################################

In [ ]:
for i, row in tqdm(df.iterrows()):
  if row['num_ingredients'] == 0 and str(row['International non-proprietary name (INN) / common name']) != 'nan':
    codes = []
    d_df = rxnorm[rxnorm.concept_name == row['International non-proprietary name (INN) / common name'].lower()]
    if d_df.shape[0] > 0:
      codes = d_df.concept_id.tolist()
      df.at[i, 'rxnorm_code'] = codes
df['num_ingredients'] = df.rxnorm_code.apply(lambda x: len(x) if x is not None else 0)
df['num_ingredients'].value_counts()

In [164]:
for i, row in tqdm(df.iterrows()):
  try:
    if row['num_ingredients'] == 0:
      drugs = row['International non-proprietary name (INN) / common name'].split(', ')
      drug_codes = []
      for drug in drugs:
        if drug in rxnorm_dict.keys():
          drug_codes.append(rxnorm_dict[drug])
        else:
          url = 'https://rxnav.nlm.nih.gov/REST/rxcui.json?name={}&search=1'.format(drug)
          j = requests.get(url).json()
          drug_codes.extend(j['idGroup']['rxnormId'])
      drug_codes = list(set(drug_codes))
      if len(drug_codes) > 0:
        df.at[i, 'rxnorm_code'] = drug_codes
  except:
      continue

1538it [00:48, 31.97it/s]


In [165]:
df['num_ingredients'] = df.rxnorm_code.apply(lambda x: len(x) if x is not None else 0)
df['num_ingredients'].value_counts()

1    1244
0     150
2     123
3      18
4       3
Name: num_ingredients, dtype: int64

In [ ]:
rxnorm = pd.read_csv(external_data_folder+'athena_rxnorm_atc/CONCEPT.csv', delimiter = '\t')
rxnorm = rxnorm[(rxnorm.domain_id == 'Drug')&(rxnorm.vocabulary_id.isin(['RxNorm', 'RxNorm Extension']))&(rxnorm.concept_class_id.str.contains('Ingredient'))]
rxnorm['concept_name'] = rxnorm['concept_name'].apply(lambda x: x.lower())
rxnorm.head(1)

In [169]:
for i, row in tqdm(df.iterrows()):
  if row['num_ingredients'] == 0 and str(row['International non-proprietary name (INN) / common name']) != 'nan':
    codes = []
    d_df = rxnorm[rxnorm.concept_name == row['International non-proprietary name (INN) / common name'].lower()]
    if d_df.shape[0] > 0:
      codes = d_df.concept_id.tolist()
      df.at[i, 'rxnorm_code'] = codes
df['num_ingredients'] = df.rxnorm_code.apply(lambda x: len(x) if x is not None else 0)
df['num_ingredients'].value_counts()

1538it [00:01, 1320.60it/s]


1    1267
2     145
0     105
3      18
4       3
Name: num_ingredients, dtype: int64

In [ ]:
rxnorm = pd.read_csv(external_data_folder+'athena_rxnorm_atc/CONCEPT.csv', delimiter = '\t')
rxnorm = rxnorm[(rxnorm.domain_id == 'Drug')&(rxnorm.vocabulary_id.isin(['RxNorm', 'RxNorm Extension']))&(rxnorm.concept_class_id.str.contains('Brand Name'))]
rxnorm['concept_name'] = rxnorm['concept_name'].apply(lambda x: x.lower())
rxnorm.head(1)

In [172]:
for i, row in tqdm(df.iterrows()):
  if row['num_ingredients'] == 0 and str(row['Medicine name']) != 'nan':
    codes = []
    d_df = rxnorm[rxnorm.concept_name == row['Medicine name'].lower()]
    if d_df.shape[0] > 0:
      codes = d_df.concept_id.tolist()
      df.at[i, 'rxnorm_code'] = codes
df['num_ingredients'] = df.rxnorm_code.apply(lambda x: len(x) if x is not None else 0)
df['num_ingredients'].value_counts()

1538it [00:02, 748.94it/s]


1    1328
2     149
0      40
3      18
4       3
Name: num_ingredients, dtype: int64

In [174]:
df.head(1)

,Medicine name,Product number,Active substance,Authorisation status,International non-proprietary name (INN) / common name,ATC code,rxnorm_code,num_ingredients
0,Steglujan,EMEA/H/C/004313,"ertugliflozin l-pyroglutamic acid, sitagliptin...",Authorised,"ertugliflozin, sitagliptin",A10BD24,"[1992672, 593411]",2


In [175]:
print(df.shape[0])
df.to_csv(data_folder+'data/ingredients.csv', index=False)

1538


----

## merge with items

In [182]:
!mkdir $data_folder'data/final_eu'

In [213]:
ingr_df = pd.read_csv(data_folder+'data/ingredients.csv')
ingr_df['Medicine name'] = ingr_df['Medicine name'].apply(lambda x: x.lower())
ingr_df['rxnorm_code'] = ingr_df['rxnorm_code'].apply(lambda x: ast.literal_eval(x) if str(x) != 'nan' else '')
ingr_df.head(1)

,Medicine name,Product number,Active substance,Authorisation status,International non-proprietary name (INN) / common name,ATC code,rxnorm_code,num_ingredients
0,steglujan,EMEA/H/C/004313,"ertugliflozin l-pyroglutamic acid, sitagliptin...",Authorised,"ertugliflozin, sitagliptin",A10BD24,"[1992672, 593411]",2


In [214]:
rxnorm = pd.read_csv(external_data_folder+'athena_rxnorm_atc/CONCEPT.csv', delimiter = '\t')
rxnorm = rxnorm[(rxnorm.domain_id == 'Drug')&(rxnorm.vocabulary_id.isin(['RxNorm', 'RxNorm Extension']))&(rxnorm.concept_class_id.str.contains('Ingredient'))]
rxnorm['concept_name'] = rxnorm['concept_name'].apply(lambda x: x.lower())
rxnorm['concept_code'] = rxnorm['concept_code'].astype(str)
rxnorm.head(1)

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
212836,40224159,alcaftadine,Drug,RxNorm,Ingredient,S,1000082,20100905,20991231,NaN


In [215]:
rxnorm_concept_rxnorm = dict(zip(rxnorm.concept_code, rxnorm.concept_id))
ingr_df['rxnorm_code'] = ingr_df['rxnorm_code'].apply(lambda x: [str(i) for i in x] if x != '' else '')
ingr_df['omop_concept_id'] = ingr_df['rxnorm_code'].apply(lambda x: [rxnorm_concept_rxnorm[i] for i in x if i in rxnorm_concept_rxnorm.keys()] if x != '' else '')

In [218]:
ingr_final = ingr_df[['Product number', 'International non-proprietary name (INN) / common name', 'rxnorm_code', 'num_ingredients', 'omop_concept_id']]
ingr_final.columns = ['product_id', 'ingredients', 'num_ingredients', 'rxnorm_id', 'omop_concept_id']
ingr_final.to_csv(data_folder+'data/final_eu/ingredients.csv', index=False)

In [236]:
ade_all_labels = ade_all_labels.merge(ingr_df, left_on = 'drug', right_on = 'Medicine name', how = 'left')
ade_all_labels = ade_all_labels[['Product number', 'Medicine name', 'International non-proprietary name (INN) / common name', 'rxnorm_code', 'pt_meddra_id', 'pt_meddra_term', 'num_ingredients', 'Authorisation status']]
ade_all_labels.columns = ['product_id', 'drug_name', 'ingredient_names', 'ingredient_codes', 'pt_meddra_id', 'pt_meddra_term', 'num_ingredients', 'auth']
ade_all_labels.head()

,product_id,drug_name,ingredient_names,ingredient_codes,pt_meddra_id,pt_meddra_term,num_ingredients,auth
0,EMEA/H/C/000690,nexavar,sorafenib,[495881],10040844,skin desquamation,1,Authorised
1,EMEA/H/C/000690,nexavar,sorafenib,[495881],10061218,inflammation,1,Authorised
2,EMEA/H/C/000690,nexavar,sorafenib,[495881],10028116,mucosal inflammation,1,Authorised
3,EMEA/H/C/000690,nexavar,sorafenib,[495881],10050537,protein,1,Authorised
4,EMEA/H/C/000690,nexavar,sorafenib,[495881],10054888,transaminases,1,Authorised


In [237]:
ade_all_labels.to_csv(data_folder+'data/final_eu/adverse_events_all_labels.csv', index=False)

In [238]:
#filter for auth
ade_all_labels_active = ade_all_labels[ade_all_labels.auth == 'Authorised']
ade_all_labels_active.to_csv(data_folder+'data/final_eu/adverse_events_active_labels.csv', index=False)

In [293]:
ade_all_labels = pd.read_csv(data_folder+'data/final_eu/adverse_events_all_labels.csv')
ade_all_labels['ingredient_codes'] = ade_all_labels.ingredient_codes.apply(lambda x: str(x).replace("'",'').replace('[','').replace(']',''))
ade_all_labels['pt_meddra_id'] = ade_all_labels['pt_meddra_id'].apply(lambda x: int(float(x)) if str(x) != 'nan' else '')
ade_all_labels.head(1)

,product_id,drug_name,ingredient_names,ingredient_codes,pt_meddra_id,pt_meddra_term,num_ingredients,auth
0,EMEA/H/C/000690,nexavar,sorafenib,495881,10040844,skin desquamation,1,Authorised


In [280]:
uniq_ingr = ade_all_labels[['drug_name', 'ingredient_names', 'ingredient_codes', 'num_ingredients']].drop_duplicates()
uniq_ingr = uniq_ingr.groupby('ingredient_codes')['drug_name'].apply(list).reset_index()
uniq_ingr['num_labels'] = uniq_ingr.drug_name.apply(lambda x: len(x))
uniq_event = ade_all_labels[['pt_meddra_id', 'pt_meddra_term']].drop_duplicates()

In [281]:
import collections
ade_count = ade_all_labels[['ingredient_codes', 'pt_meddra_id']].groupby('ingredient_codes')['pt_meddra_id'].apply(list).reset_index()
ade_count['pt_meddra_list'] = ade_count['pt_meddra_id'].apply(lambda x: collections.Counter(x))
ade_count['pt_meddra_id'] = ade_count['pt_meddra_list'].apply(lambda x: x.keys())
ade_count = ade_count.explode('pt_meddra_id')
ade_count['num_events'] = ade_count.apply(lambda x: x['pt_meddra_list'][x['pt_meddra_id']], axis = 1)

In [288]:
ades = uniq_ingr.merge(ade_count[['ingredient_codes', 'pt_meddra_id', 'num_events']], on = 'ingredient_codes', how = 'left')
ades['pct_labels'] = ades.apply(lambda x: x['num_events'] / x['num_labels'], axis = 1)
ades = ades.merge(uniq_event, on = 'pt_meddra_id', how = 'left')

In [300]:
a = ade_all_labels.merge(ades[['ingredient_codes', 'pt_meddra_id', 'pct_labels', 'num_labels', 'num_events']], on = ['ingredient_codes', 'pt_meddra_id'], how = 'left')
a = a.drop(['drug_name', 'product_id', 'auth'], axis = 1).drop_duplicates()
a.to_csv(data_folder+'data/final_eu/adverse_events.csv', index=False)

In [299]:
a.drop(['drug_name', 'product_id', 'auth'], axis = 1)

,ingredient_names,ingredient_codes,pt_meddra_id,pt_meddra_term,num_ingredients,pct_labels,num_labels,num_events
13,sorafenib,495881,10054888,transaminases,1,1.0,2,2
14,sorafenib,495881,10054888,transaminases,1,1.0,2,2
21,sorafenib,495881,10000496,acne,1,1.0,2,2
38,sorafenib,495881,10050659,lipase,1,1.0,2,2
39,sorafenib,495881,10050659,lipase,1,1.0,2,2
...,...,...,...,...,...,...,...,...
176994,idarucizumab,1716191,10040722,Single photon emission computerised tomogram,1,1.0,1,1
176995,idarucizumab,1716191,10042609,Surgery,1,1.0,1,1
176996,idarucizumab,1716191,10067484,Adverse reaction,1,1.0,1,1
176997,idarucizumab,1716191,10067484,Adverse reaction,1,1.0,1,1


---
## notes



In [136]:
#rxnorm-atc mapping
atc = pd.read_csv(external_data_folder+'athena_rxnorm_atc/CONCEPT.csv', delimiter = '\t')
atc = atc[atc.concept_code.isin(df[df['num_ingredients'] == 0]['ATC code'].unique().tolist())][['concept_id', 'concept_name', 'concept_code']].drop_duplicates()
atc.columns = ['atc_{}'.format(i) for i in atc.columns]
atc.head()

<ipython-input-136-fdf7d1ff8510>:2: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  atc = pd.read_csv(external_data_folder+'athena_rxnorm_atc/CONCEPT.csv', delimiter = '\t')


,atc_concept_id,atc_concept_name,atc_concept_code
491,21600696,rimonabant; oral,A08AX01
496,21600701,"multienzymes (lipase, protease etc.)",A09AA02
602,21600784,sitagliptin; oral,A10BH01
792,21600941,Enzymes,A16AB
953,21601062,"coagulation factor VIII; parenteral, topical, ...",B02BD02


In [130]:
rel = pd.read_csv(external_data_folder+'athena_rxnorm_atc/CONCEPT_RELATIONSHIP.csv', delimiter = '\t')
rel = rel[(rel.concept_id_1.isin(atc.concept_id))&(rel.concept_id_2.isin(rxnorm.concept_id))&(rel.relationship_id.str.contains('ATC - RxNorm'))]
rel = rel[['concept_id_1', 'concept_id_2']].merge(rxnorm[['concept_id', 'concept_name', 'concept_class_id', 'concept_code']], left_on = 'concept_id_2', right_on = 'concept_id', how = 'left')
rel = rel.merge(atc, left_on = 'concept_id_1', right_on = 'atc_concept_id', how = 'left')
rel = rel.groupby('atc_concept_code')['concept_id'].apply(list).reset_index()

In [148]:
rel.head(1)

,atc_concept_code,concept_id
0,A08AX01,[36878933]


In [149]:
rel_dict = dict(zip(rel.atc_concept_code, rel.concept_id))
df['rxnorm_code'] = df.apply(lambda x: rel_dict[x['ATC code']]
                             if x['num_ingredients'] == 0 and str(x['ATC code']) != 'nan' and x['ATC code'] in rel_dict.keys()
                             else x['rxnorm_code'], axis = 1)
df['num_ingredients'] = df.rxnorm_code.apply(lambda x: len(x) if x is not None else 0)
df['num_ingredients'].value_counts()

1      1289
2       125
0        46
3        22
20       14
4         7
10        4
13        4
7         4
17        3
5         3
6         3
31        3
232       3
9         2
12        2
8         1
50        1
23        1
28        1
Name: num_ingredients, dtype: int64

In [173]:
df[df['num_ingredients'] == 0]

,Medicine name,Product number,Active substance,Authorisation status,International non-proprietary name (INN) / common name,ATC code,rxnorm_code,num_ingredients
22,Abrysvo,EMEA/H/C/006027,"Respiratory syncytial virus, subgroup B, stabi...",Authorised,Respiratory syncytial virus vaccines,J07BX05,[],0
28,Spikevax (previously COVID-19 Vaccine Moderna),EMEA/H/C/005791,"Single-stranded, 5’-capped messenger RNA (mRNA...",Authorised,"elasomeran, elasomeran / imelasomeran, elasome...",J07BN01,[],0
85,Bimervax,EMEA/H/C/006058,SARS-CoV-2 virus recombinant spike (S) protein...,Authorised,"COVID-19 Vaccine (recombinant, adjuvanted)",J07BN,[],0
121,Nuvaxovid,EMEA/H/C/005808,SARS-CoV-2 recombinant spike protein,Authorised,"COVID-19 Vaccine (recombinant, adjuvanted)",J07BX03,[],0
157,Vaxzevria (previously COVID-19 Vaccine AstraZe...,EMEA/H/C/005675,ChAdOx1-SARS-COV-2,Authorised,COVID-19 Vaccine (ChAdOx1-S [recombinant]),J07BN02,[],0
166,Zabdeno,EMEA/H/C/005337,Recombinant Adenovirus type 26 (Ad26) encoding...,Authorised,ebola vaccine (Ad26.ZEBOV-GP [recombinant]),J07,[],0
167,Mvabea,EMEA/H/C/005343,Recombinant Modified Vaccinia Ankara Bavarian ...,Authorised,Ebola vaccine (MVA-BN-Filo [recombinant]),J07BX,[],0
184,Flucelvax Tetra,EMEA/H/C/004814,A/Darwin/6/2021(H3N2)-like strain (A/Darwin/11...,Authorised,"influenza vaccine (surface antigen, inactivate...",J07BB02,[],0
196,Fluad Tetra,EMEA/H/C/004993,A/Darwin/9/2021 (H3N2) - like strain (A/Darwin...,Authorised,"influenza vaccine (surface antigen, inactivate...",J07BB02,[],0
371,Jcovden (previously COVID-19 Vaccine Janssen),EMEA/H/C/005737,adenovirus type 26 encoding the SARS-CoV-2 spi...,Authorised,COVID-19 vaccine (Ad26.COV2-S [recombinant]),J07BN02,[],0
